In [55]:
import sys
sys.path.append(r'E:/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [116]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import gensim
import re
import smart_open
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.classification import LogisticClassifier
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.classification import precision_score
from pyml.model_selection import KFold
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据集

1. 读取训练数据和测试数据为字符串的列表
2. 读取训练集label，并转换为数字格式

In [106]:
def read_train_text_to_list(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        contents = f.readlines()
    lines = [l.strip() for l in contents]
    return lines

In [107]:
# train_ori_X = read_train_text_to_list('../data/trainData.txt')
train_ori_Y = read_train_text_to_list('../data/trainLabel.txt')
train_ori_Y = np.array([int(y) for y in train_ori_Y])
# test_ori_X = read_train_text_to_list('../data/testData.txt')

In [81]:
def read_raw_documents(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="UTF-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                # 变小写，去标点符号，分词
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [89]:
train_sentences = list(read_raw_documents('../data/trainData.txt'))
test_sentences = list(read_raw_documents('../data/testData.txt', tokens_only=True))

# 数据预处理 & 特征工程
1. Count Vectors as feature
2. TF-IDF Vectors as festures
3. Word Embeddings as features
4. Text/NLP based features
5. Topic Models as features

In [83]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [84]:
model.build_vocab(train_sentences)

In [85]:
model.train(train_sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [100]:
n_train_samples = len(train_sentences)
n_test_samples = len(test_sentences)
vector_size = 50
train_X = np.zeros((n_train_samples, vector_size))
test_X = np.zeros((n_test_samples, vector_size))
for i in range(0, n_train_samples):
    train_X[i] = model.infer_vector(train_sentences[i][0])
for i in range(0, n_test_samples):
    test_X[i] = model.infer_vector(test_sentences[i])

In [109]:
train_Y = train_ori_Y

In [111]:
train_Y.shape

(24000,)

# 交叉验证-寻找最好的参数

# 训练模型

In [118]:
clf = LogisticClassifier(learning_rate=0.1, num_iterations=30000)
clf.fit(train_X, train_Y, watch=True)

Cost after iteration 0: 1.483168
Cost after iteration 100: 0.805138
Cost after iteration 200: 0.640280
Cost after iteration 300: 0.560042
Cost after iteration 400: 0.514352
Cost after iteration 500: 0.485844
Cost after iteration 600: 0.467064
Cost after iteration 700: 0.454278
Cost after iteration 800: 0.445383
Cost after iteration 900: 0.439109
Cost after iteration 1000: 0.434645
Cost after iteration 1100: 0.431454
Cost after iteration 1200: 0.429164
Cost after iteration 1300: 0.427517
Cost after iteration 1400: 0.426329
Cost after iteration 1500: 0.425469
Cost after iteration 1600: 0.424844
Cost after iteration 1700: 0.424387
Cost after iteration 1800: 0.424050
Cost after iteration 1900: 0.423800
Cost after iteration 2000: 0.423611
Cost after iteration 2100: 0.423466
Cost after iteration 2200: 0.423353
Cost after iteration 2300: 0.423264
Cost after iteration 2400: 0.423191
Cost after iteration 2500: 0.423130
Cost after iteration 2600: 0.423079
Cost after iteration 2700: 0.423034
Cost

Cost after iteration 22500: 0.422225
Cost after iteration 22600: 0.422225
Cost after iteration 22700: 0.422225
Cost after iteration 22800: 0.422225
Cost after iteration 22900: 0.422225
Cost after iteration 23000: 0.422225
Cost after iteration 23100: 0.422225
Cost after iteration 23200: 0.422225
Cost after iteration 23300: 0.422225
Cost after iteration 23400: 0.422225
Cost after iteration 23500: 0.422225
Cost after iteration 23600: 0.422225
Cost after iteration 23700: 0.422225
Cost after iteration 23800: 0.422225
Cost after iteration 23900: 0.422225
Cost after iteration 24000: 0.422225
Cost after iteration 24100: 0.422225
Cost after iteration 24200: 0.422225
Cost after iteration 24300: 0.422225
Cost after iteration 24400: 0.422225
Cost after iteration 24500: 0.422225
Cost after iteration 24600: 0.422225
Cost after iteration 24700: 0.422225
Cost after iteration 24800: 0.422225
Cost after iteration 24900: 0.422225
Cost after iteration 25000: 0.422225
Cost after iteration 25100: 0.422225
C

In [114]:
# k_range = range(1,21)
# k_splits = 10
# ms = KFold(k_splits=k_splits)
# k_scores = np.zeros((len(k_range)))
# for train_indices, test_indices in ms.split(train_X):
#     for k in k_range:
#         clf = KNeighborsClassifier(k=k)
#         clf.fit(train_X[train_indices], train_Y[train_indices])
#         y_pred = clf.predict(train_X[test_indices])
#         score = precision_score(train_Y[test_indices], y_pred)
# #         print('k : {} score: {}'.format(k, score))
#         k_scores[k_range.index(k)] += score
# avg_k_scores = k_scores/k_splits
# print(avg_k_scores)
# print("best k ", np.argmax(avg_k_scores)+1)

# 训练模型写入结果

In [120]:
y_pred = clf.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+'doc2vec-lr-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)